In [1]:
from ephys_queries import select_spike_times
from ephys_queries import db_setup_core
from dotenv import load_dotenv
from pathlib import Path

In [2]:
from spiketimes.df.binning import binned_spiketrain_bins_provided
from spiketimes.df.statistics import auc_roc_test_by
from spiketimes.df.statistics import diffmeans_test_by
from spiketimes.df.baseline import zscore_standardise_by
from spiketimes.utils import p_adjust

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import auc

In [4]:
data_dir = Path(".").absolute().parent / "data"
fig_dir = Path(".").absolute().parent / "figs"

In [5]:
group_names = ["acute_citalopram", 
            "acute_saline", 
            "acute_cit", 
            "acute_sal"]
dfb = (
    pd.read_csv(data_dir / "baseline.csv")
    .loc[lambda x: x["group_name"].isin(group_names)]
)

In [6]:
block_name = "chal"
t_before = 10 * 60
fs = 30000
load_dotenv()
engine, metadata = db_setup_core()

df_spikes = (
    select_spike_times(
            engine, 
            metadata,
            group_names=group_names,
            block_name=block_name,
        t_before=t_before,
        align_to_block=True
            )
    .assign(spiketimes=lambda x: x["spike_time_samples"].divide(fs))
)

In [7]:
df_spikes.head(3)

,neuron_id,spike_time_samples,group_name,spiketimes
0,1011,-17996636,acute_cit,-599.887867
1,1011,-17992217,acute_cit,-599.740567
2,1011,-17987075,acute_cit,-599.569167


### Define Responders

In [8]:
def response_dm(row):
    if row["p_dm"] >= 0.05:
        return "Non Responder"
    elif row["diff_of_means"] < 0:
        return "Inhibited"
    else:
        return "Stimulated"
    
    
def response_auc(row):
    if row["p_dm"] >= 0.05:
        return "Non Responder"
    elif row["AUCROC"] < 0.5:
        return "Inhibited"
    else:
        return "Stimulated"

In [9]:
bins = np.arange(-600, 1260, 120)

df = (
    binned_spiketrain_bins_provided(
        df_spikes, 
        bins, 
        spiketimes_col="spiketimes",
                               
        spiketrain_col="neuron_id"
    )
    .rename(columns={"counts": "spike_counts", "bin": "time"})
    .merge(dfb[["neuron_id", "cluster", "group_name"]])
    .assign(treatment=lambda x: x["group_name"].map({
                        "acute_cit": "Citalopram",
                        "acute_citalopram": "Citalopram",
                        "acute_saline": "Saline",
                        "acute_sal": "Saline"}),
           condition=lambda x: x["time"].apply(lambda y: "Baseline" if y < 0 else "Challenge"),
           firing_rate=lambda x: x["spike_counts"].divide(60),
           cond=lambda x: x["condition"].map({"Baseline":0, "Challenge":1})
        )
    .drop("group_name", axis=1)
)

res_auc = auc_roc_test_by(df, n_boot=1000, 
                      spiketrain_col="neuron_id", 
                      spikecount_col="firing_rate",
                     condition_col="cond")
res_dm = diffmeans_test_by(df, n_boot=5000, 
                      spiketrain_col="neuron_id", 
                      spikecount_col="firing_rate",
                     condition_col="cond")

In [10]:
res_auc = auc_roc_test_by(df, n_boot=1000, 
                      spiketrain_col="neuron_id", 
                      spikecount_col="firing_rate",
                     condition_col="cond")

In [11]:
res_dm = diffmeans_test_by(df, n_boot=5000, 
                      spiketrain_col="neuron_id", 
                      spikecount_col="firing_rate",
                     condition_col="cond")

In [12]:
res_auc["p_AUCROC"] = p_adjust(res_auc["p"])
res_dm["p_dm"] = p_adjust(res_dm["p"])
df_effects = (
    res_auc.merge(df[["neuron_id", "cluster", "treatment"]]
              .drop_duplicates())
    .drop("p", axis=1)
    .merge(res_dm)
    .drop("p", axis=1)
    .assign(
        response_dm=lambda x: x.apply(response_dm, axis=1),
        response_aucroc=lambda x: x.apply(response_auc, axis=1))
)

### Get Z Score Time Series

In [13]:
bins = np.arange(-600, 1200, 10)
df_zscores = (
    binned_spiketrain_bins_provided(df_spikes, bins=bins, spiketimes_col="spiketimes", spiketrain_col="neuron_id")
    .merge(df_effects[["response_dm", "cluster", "treatment", "neuron_id"]])
    .pipe(lambda x: zscore_standardise_by(df=x, baseline_start_stop=np.array([-600, 0]),
                     spiketrain_col="neuron_id", data_col="counts", time_col="bin"))
)

In [14]:
df_effects = (
    df_zscores.groupby("neuron_id")
    .apply(lambda x: auc(np.arange(len(x)), x["zscore"]))
    .reset_index()
    .rename(columns={0:"auc"})
    .merge(df_effects[["neuron_id", "cluster", "treatment", "response_dm", "response_aucroc"]])
)

In [15]:
df_effects.to_csv(data_dir / "cit_effects.csv", index=False)
df_zscores.to_csv(data_dir / "cit_zscores.csv", index=False)